In [ ]:
from dataclasses import dataclass

@dataclass
class Config:
    # Lattice sizes
    Lt: int = 4
    Lx: int = 8
    Ly: int = 8
    # Gauge group SU(N)
    N: int = 3
    # Couplings and anisotropy
    beta: float = 6.0
    at: float = 1.0
    a: float = 1.0
    # Orbifold mass terms
    m2: float = 50.0            # radial penalty
    m2_u1: float = 50.0         # U(1) penalty
    # HMC
    ntau: int = 10
    dtau_t: float = 0.02        # time-link step
    dtau_s: float = 0.02        # space-link step
    n_therm: int = 100
    n_traj: int = 1000
    nskip: int = 10
    # Init and RNG
    init: int = 1               # 1=random, 0=unit
    seed: int = 1234
    # Logging
    log_interval: int = 10


In [ ]:
import numpy as np
from dataclasses import dataclass

@dataclass
class Lattice:
    Lt: int
    Lx: int
    Ly: int

    def shape_sites(self):
        return (self.Lt, self.Lx, self.Ly)

    def shift(self, arr, axis, step):
        return np.roll(arr, shift=step, axis=axis)

In [ ]:
import numpy as np

def random_suN(N, rng):
    X = rng.normal(size=(N, N)) + 1j * rng.normal(size=(N, N))
    Q, R = np.linalg.qr(X)
    # make det=1
    det = np.linalg.det(Q)
    Q *= det ** (-1.0 / N)
    return Q

def init_fields(cfg, lat, rng):
    # umat[t, x, y] in SU(N)
    umat = np.empty(lat.shape_sites(), dtype=object)
    for it in range(cfg.Lt):
        for ix in range(cfg.Lx):
            for iy in range(cfg.Ly):
                umat[it, ix, iy] = random_suN(cfg.N, rng) if cfg.init == 1 else np.eye(cfg.N, dtype=complex)
    # zmat[j, t, x, y] as complex matrices (j=0:x,1:y)
    zmat = np.empty((2,) + lat.shape_sites(), dtype=object)
    for j in range(2):
        for it in range(cfg.Lt):
            for ix in range(cfg.Lx):
                for iy in range(cfg.Ly):
                    # complex noncompact init
                    Z = (rng.normal(size=(cfg.N, cfg.N)) + 1j * rng.normal(size=(cfg.N, cfg.N))) / np.sqrt(2.0)
                    if cfg.init == 0:
                        Z = np.eye(cfg.N, dtype=complex)
                    zmat[j, it, ix, iy] = Z
    return umat, zmat


In [ ]:
import numpy as np

def radial_penalty(Z):
    # Tr[(Z^\dagger Z - I)^2]
    N = Z.shape[0]
    H = Z.conj().T @ Z - np.eye(N, dtype=complex)
    return np.real(np.trace(H.conj().T @ H))

def u1_penalty(Z):
    # (|det U| - 1)^2 via polar unitary U ≈ Z (Z^\dagger Z)^{-1/2}
    # numerically, penalize |det(Z)| away from 1 as a proxy
    detZ = np.linalg.det(Z)
    return (np.abs(detZ) - 1.0) ** 2

def s_mass(zmat, cfg):
    val = 0.0
    for j in range(2):
        it_max, ix_max, iy_max = zmat.shape[1:4]
        for it in range(it_max):
            for ix in range(ix_max):
                for iy in range(iy_max):
                    Z = zmat[j, it, ix, iy]
                    val += cfg.m2 * radial_penalty(Z)
                    val += cfg.m2_u1 * u1_penalty(Z)
    return val

def s_kin(umat, zmat, lat, cfg):
    # || U_t(t,x,y) Z_j(t+1,x,y) - Z_j(t,x,y) U_t(t,x+e_j,y+e_j?) ||^2
    # Here we use a simple gauge-covariant nearest-neighbor difference in t
    val = 0.0
    for j in range(2):
        Z = zmat[j]
        Z_fwd_t = np.roll(Z, shift=-1, axis=1)  # forward in t
        it_max, ix_max, iy_max = Z.shape[1:4]
        for it in range(it_max):
            for ix in range(ix_max):
                for iy in range(iy_max):
                    Ut = umat[it, ix, iy]
                    term = Ut @ Z_fwd_t[j, it, ix, iy] - Z[j, it, ix, iy] @ Ut
                    val += np.real(np.trace(term.conj().T @ term))
    return (cfg.beta * cfg.at / cfg.a) * val

def s_spa(zmat, lat, cfg):
    # Plaquette-like: || Z_x Z_y(x+ex) - Z_y Z_x(x+ey) ||^2
    val = 0.0
    Zx = zmat[0]
    Zy = zmat[1]
    Zx_fwd_x = np.roll(Zx, shift=-1, axis=2)  # +x
    Zy_fwd_y = np.roll(Zy, shift=-1, axis=3)  # +y
    it_max, ix_max, iy_max = Zx.shape[1:4]
    for it in range(it_max):
        for ix in range(ix_max):
            for iy in range(iy_max):
                A = Zx[0, it, ix, iy] @ Zy_fwd_y[1, it, ix, iy]
                B = Zy[1, it, ix, iy] @ Zx_fwd_x[0, it, ix, iy]
                term = A - B
                val += np.real(np.trace(term.conj().T @ term))
    return (cfg.beta * cfg.a / cfg.at) * val

def action_total(umat, zmat, lat, cfg):
    return s_kin(umat, zmat, lat, cfg) + s_spa(zmat, lat, cfg) + s_mass(zmat, cfg)


In [ ]:
import numpy as np
from copy import deepcopy
from .action import action_total

def local_update_matrix(mat, amplitude, rng):
    # Add a small random complex perturbation to a matrix
    perturb = amplitude * (rng.normal(size=mat.shape) + 1j * rng.normal(size=mat.shape))
    return mat + perturb

def metropolis_update(umat, zmat, lat, cfg, rng):
    accept = 0
    total = 0

    # Update time links (umat)
    it_max, ix_max, iy_max = umat.shape
    for it in range(it_max):
        for ix in range(ix_max):
            for iy in range(iy_max):
                old = umat[it, ix, iy]
                umat_new = deepcopy(umat)
                umat_new[it, ix, iy] = local_update_matrix(old, amplitude=0.10, rng=rng)  # Tune amplitude
                dS = action_total(umat_new, zmat, lat, cfg) - action_total(umat, zmat, lat, cfg)
                if np.log(rng.uniform()) < -dS:
                    umat[it, ix, iy] = umat_new[it, ix, iy]
                    accept += 1
                total += 1

    # Update space links (zmat)
    for j in range(2):
        it_max, ix_max, iy_max = zmat.shape[1:4]
        for it in range(it_max):
            for ix in range(ix_max):
                for iy in range(iy_max):
                    old = zmat[j, it, ix, iy]
                    zmat_new = deepcopy(zmat)
                    zmat_new[j, it, ix, iy] = local_update_matrix(old, amplitude=0.10, rng=rng)
                    dS = action_total(umat, zmat_new, lat, cfg) - action_total(umat, zmat, lat, cfg)
                    if np.log(rng.uniform()) < -dS:
                        zmat[j, it, ix, iy] = zmat_new[j, it, ix, iy]
                        accept += 1
                    total += 1

    acc_rate = accept / total
    return umat, zmat, acc_rate


In [ ]:
import numpy as np
from src.config import Config
from src.lattice import Lattice
from src.fields import init_fields
from src.metropolis import metropolis_update
from src.measure import polyakov_loop_abs, radial_monitor, u1_monitor, plaquette_like_energy

def run():
    cfg = Config()
    rng = np.random.default_rng(cfg.seed)
    lat = Lattice(cfg.Lt, cfg.Lx, cfg.Ly)
    umat, zmat = init_fields(cfg, lat, rng)

    # 热化
    n_therm_steps = cfg.n_therm
    for step in range(n_therm_steps):
        umat, zmat, acc_rate = metropolis_update(umat, zmat, lat, cfg, rng)

    for traj in range(1, cfg.n_traj + 1):
        umat, zmat, acc_rate = metropolis_update(umat, zmat, lat, cfg, rng)
        if traj % cfg.log_interval == 0:
            print(f"[step={traj}] acc={acc_rate:.2f}")

        if traj % cfg.nskip == 0:
            P = polyakov_loop_abs(umat)
            R = radial_monitor(zmat)
            U1 = u1_monitor(zmat)
            Epl = plaquette_like_energy(zmat)
            print(f"meas: |P|={P:.4f}  radial={R:.4e}  u1={U1:.4f}  Epl={Epl:.4e}")

if __name__ == "__main__":
    run()
